In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import numpy as np

In [2]:
# Device configuration
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda')

In [3]:
# Hyper-parameters
num_epochs = 80
in_channels = 30
num_classes = 600
batch_size = 500
learning_rate = 0.00003

In [4]:
# Data
#np.save('X_train.npy', X_train)
X_train_load = np.load('X_train.npy')

#np.save('y_train.npy', y_train)
y_train_load = np.load('y_train.npy')

#np.save('X_test.npy', X_test)
X_test_load = np.load('X_test.npy')

#np.save('y_test.npy', y_test)
y_test_load = np.load('y_test.npy')

In [5]:
X_test_load.shape

(163, 64, 64, 30)

In [6]:
class MedDataset(data.Dataset):
    def __init__(self, features, labels):
        self.features = features.swapaxes(1,3)
        
        self.labels = labels
    
    def __getitem__(self, index):
        x, y = self.features[index] , self.labels[index]
        return x, y
    
    def __len__(self):
        return len(self.features)

train_data = MedDataset(X_train_load, y_train_load)
test_data = MedDataset(X_test_load, y_test_load)

In [7]:
train_loader = torch.utils.data.DataLoader(dataset = train_data,
                                           batch_size = batch_size,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_data,
                                          batch_size = 100,
                                          shuffle = False)

In [8]:
def conv3x3(in_channels, out_channels, stride = 1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                    stride = stride, padding = 1, bias = False)

In [9]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace = True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [10]:
# ResNet
class ResNet(nn.Module):
    def __init__(self, in_channels, block, layers, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = in_channels
        self.conv = conv3x3(30, 30)
        self.bn = nn.BatchNorm2d(30)
        self.relu = nn.ReLU(inplace = True)
        self.layer1 = self.make_layer(block, 30, 64, layers[0])
        self.layer2 = self.make_layer(block, 64, 64, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, 128, layers[2], 2)
        self.layer4 = self.make_layer(block, 128, 256, layers[3], 2)
        self.layer5 = self.make_layer(block, 256, 512, layers[4], 2)
        self.layer6 = self.make_layer(block, 512, 512, layers[5], 2)
        self.avg_pool = nn.AvgPool2d(8)
        #self.conv2 = conv3x3(64,32)
        self.fc = nn.Linear(512, num_classes)
    
    def make_layer(self, block, in_channels, out_channels, blocks, stride = 1):
        downsample = None
        if (stride != 1) or (in_channels != out_channels):
            downsample = nn.Sequential(
            conv3x3(in_channels, out_channels, stride = stride),
            nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(in_channels, out_channels, stride, downsample))
        in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
            return nn.Sequential(*layers)
        
    def forward(self, x):
        x = x.float()
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.avg_pool(x)
        x = self.layer2(x)
        x = self.avg_pool(x)
        x = self.layer3(x)
        x = self.avg_pool(x)
        x = self.layer4(x)
        x = self.avg_pool(x)
        x = self.layer5(x)
        x = self.avg_pool(x)
        x = self.layer6(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [11]:
import torch.nn.init as init


def weight_init(m):
    '''
    Usage:
        model = Model()
        model.apply(weight_init)
    '''
    if isinstance(m, nn.Conv1d):
        init.normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.Conv2d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.Conv3d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.ConvTranspose1d):
        init.normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.ConvTranspose2d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.ConvTranspose3d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.BatchNorm1d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm2d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm3d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        init.xavier_normal_(m.weight.data)
        init.normal_(m.bias.data)
    elif isinstance(m, nn.LSTM):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)
    elif isinstance(m, nn.LSTMCell):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)
    elif isinstance(m, nn.GRU):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)
    elif isinstance(m, nn.GRUCell):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)

In [12]:
model = ResNet(in_channels, ResidualBlock, [2,2,2,2,2,2], num_classes).to(device)
#model.apply(weight_init)

In [13]:
# Loss and optimizer
criterion = nn.MSELoss(reduction = 'sum')
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

# For updating learning rate
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (X, y) in enumerate(train_loader):
        X = X.to(device)
        y = y.to(device).float()
        #print(X.shape)
        
        # Forward pass
        outputs = model(X)
        loss = criterion(outputs, y)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 1 == 0:
            print ("Epoch [{}/{}], Step [{}/{}], Loss: {:.7f}"
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()/330))
            
    # Decay learning rate
    if (epoch + 1) % 20 == 0:
        curr_lr /= 1
        update_lr(optimizer, curr_lr)


Epoch [1/80], Step [1/1], Loss: 110.4457505
Epoch [2/80], Step [1/1], Loss: 91.1557292
Epoch [3/80], Step [1/1], Loss: 36.7564157
Epoch [4/80], Step [1/1], Loss: 3.2678833
Epoch [5/80], Step [1/1], Loss: 1.3855586
Epoch [6/80], Step [1/1], Loss: 1.1456214
Epoch [7/80], Step [1/1], Loss: 1.0826960
Epoch [8/80], Step [1/1], Loss: 1.0601135
Epoch [9/80], Step [1/1], Loss: 1.0505747
Epoch [10/80], Step [1/1], Loss: 1.0466340
Epoch [11/80], Step [1/1], Loss: 1.0435207
Epoch [12/80], Step [1/1], Loss: 1.0405373
Epoch [13/80], Step [1/1], Loss: 1.0381652
Epoch [14/80], Step [1/1], Loss: 1.0360132
Epoch [15/80], Step [1/1], Loss: 1.0331344
Epoch [16/80], Step [1/1], Loss: 1.0294889
Epoch [17/80], Step [1/1], Loss: 1.0983928
Epoch [18/80], Step [1/1], Loss: 1.0299992
Epoch [19/80], Step [1/1], Loss: 1.0282264
Epoch [20/80], Step [1/1], Loss: 1.0267476
Epoch [21/80], Step [1/1], Loss: 1.0259786
Epoch [22/80], Step [1/1], Loss: 1.0248241
Epoch [23/80], Step [1/1], Loss: 1.0234937
Epoch [24/80], S

In [14]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    loss = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device).long()
        outputs = model(images)
        loss += criterion(outputs, labels.float()) 
        _, predicted = torch.max(outputs.data, 1)
        total += labels.shape[0]
        labels_new = torch.max(labels, 1)[1]
        correct += (predicted == labels_new).sum().item()
        
    print('Accuracy of the model on the test images: {} %, Loss: {}'
          .format(100 * correct / total, loss/163))


Accuracy of the model on the test images: 2.4539877300613497 %, Loss: 1.049238920211792


In [15]:
# Save the model checkpoint
torch.save(model.state_dict(), 'resnet.ckpt')

In [20]:
from tensorboardX import SummaryWriter
dummy_input = torch.rand((100,30,64,64))
with SummaryWriter(comment = 'ResNet') as writer:
    writer.add_graph(model,(X, ))

In [16]:
predicted

tensor([ 64,  64,  64, 330,  64,  64,  51,  64,  64,  64,  96,  64,  64,  64,
         64,  64,  64,  64,  64,  64,  64,  64,  64, 427,  64,  64,  64,  64,
         64,  64,  64,  64,  64,  64,  64,  64,  64,  64,  64,  64,  64,  64,
         64,  64,  64,  64,  64,  64,  64,  51,  64,  64,  64,  64,  64,  64,
         64,  64,  64,  64,  64,  64,  64], device='cuda:0')